In [ ]:
import numpy as np

def AdjointMatrix(A):
    return np.conjugate(A.T)

def BackSubstitution(R, y):
    """Given a square upper triangular matrix R and a vector y of same size this
       function solves R*x=y using backward substitution and returns x."""
    n = R.shape[1]

    # y als Spaltenvektor
    y = np.asarray(y)
    if y.ndim == 1:
        y = y.reshape(n, 1)

    x = np.zeros((n, 1), dtype=R.dtype)

    # Rückwärtseinsetzen
    for i in range(n-1, -1, -1):
        if i < n-1:
            s = np.dot(R[i, i+1:], x[i+1:, 0])
        else:
            s = 0.0
        x[i, 0] = (y[i, 0] - s) / R[i, i]

    return x


def LeastSquares(Q, R, b):
    """Given a matrix A and a vector b this function solves the least squares
       problem of minimizing ||A*x-b|| and returns the optimal x.
       Assumes that A = Q @ R and Q is unitary."""
    b = np.asarray(b)
    if b.ndim == 1:
        b = b.reshape(-1, 1)

    # Minimierung von ||Ax-b||: löse R x = Q* b
    y = AdjointMatrix(Q) @ b
    x = BackSubstitution(R, y)
    return x


In [ ]:
def LoadQRData():
    """Load the decomposed matrix data from files"""
    data_dir = "data/"

    # Standard: kleine schlecht konditionierte 3x3-Matrix
    prefix = "epsilon_matrix"
    # Für andere Matrizen einfach prefix auf
    # "lsq_ungleichung" oder "soft_body" ändern.

    A     = np.load(data_dir + prefix + "_A.npy")
    Q_gs  = np.load(data_dir + prefix + "_Q_gs.npy")
    R_gs  = np.load(data_dir + prefix + "_R_gs.npy")
    Q_mgs = np.load(data_dir + prefix + "_Q_mgs.npy")
    R_mgs = np.load(data_dir + prefix + "_R_mgs.npy")
    Q_hh  = np.load(data_dir + prefix + "_Q_hh.npy")
    R_hh  = np.load(data_dir + prefix + "_R_hh.npy")

    return A, Q_gs, R_gs, Q_mgs, R_mgs, Q_hh, R_hh


In [ ]:
if __name__=="__main__":
    A, Q_gs, R_gs, Q_mgs, R_mgs, Q_hh, R_hh = LoadQRData()

    print(f"\nIst Q wirklich orthogonal?")
    print(f"GS:  {np.linalg.norm(AdjointMatrix(Q_gs)  @ Q_gs  - np.eye(Q_gs .shape[1])):.3e}")
    print(f"MGS: {np.linalg.norm(AdjointMatrix(Q_mgs) @ Q_mgs - np.eye(Q_mgs.shape[1])):.3e}")
    print(f"HH:  {np.linalg.norm(AdjointMatrix(Q_hh)  @ Q_hh  - np.eye(Q_hh .shape[1])):.3e}")

    # Try solving a least squares system
    # b = np.array([3.0, 3.0, 3.0]).T
    b = np.ones(A.shape[1]).T
    x_gs  = LeastSquares(Q_gs , R_gs , b).flatten()
    x_mgs = LeastSquares(Q_mgs, R_mgs, b).flatten()
    x_hh  = LeastSquares(Q_hh , R_hh , b).flatten()
    x = np.linalg.lstsq(A, b, rcond=-1)[0]

    print(f"\nWie präzise sind die Lösungen eines least squares Problems damit?")
    print(f"GS:  {np.linalg.norm(x_gs  - x):.3e}")
    print(f"MGS: {np.linalg.norm(x_mgs - x):.3e}")
    print(f"HH:  {np.linalg.norm(x_hh  - x):.3e}")

In [ ]:
Der Householder-Algorithmus liefert die stabilste QR-Zerlegung und die genauesten Least-Squares-Lösungen; klassischer Gram-Schmidt ist numerisch am instabilsten,
modifizierter Gram-Schmidt liegt dazwischen (oft nah an Householder, aber nicht immer - bei der
ϵ-Matrix versagt er auch deutlich).